In [1]:
import wrds
import pandas as pd
from datetime import datetime

In [2]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
Loading library list...
Done


In [3]:
constituents = db.get_table("comp_na_daily_all", "idxcst_his").set_index("gvkey")
security = db.get_table("comp_na_daily_all", "security").set_index("gvkey")

In [4]:
data = constituents.join(security[["tic", "cusip", "dldtei"]])
data = data[data["gvkeyx"] == "000208"].reset_index() # 208=Nasdaq 100

In [5]:
dates = pd.date_range(data["from"].min(), datetime.today())

In [6]:
components = pd.DataFrame(index=dates, columns=data["gvkey"].unique())

In [7]:
for i in data.index:
    components.loc[data.loc[i, "from"]:data.loc[i, "thru"], data.loc[i, "gvkey"]] = True
components = components.asfreq("BM", how="last").dropna(axis=1, how='all')
components = components.reindex(dates, method='pad').dropna(axis=1, how='all').dropna(how='all')

In [8]:
components

,001013,001045,001055,001111,001161,001164,001254,001300,001327,001440,...,180405,180652,180711,183377,184725,184996,199356,205692,211009,212340
1995-01-31,True,NaN,True,NaN,NaN,True,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-02-01,True,NaN,True,NaN,NaN,True,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-02-02,True,NaN,True,NaN,NaN,True,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-02-03,True,NaN,True,NaN,NaN,True,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-02-04,True,NaN,True,NaN,NaN,True,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-23,NaN,NaN,NaN,NaN,True,NaN,NaN,True,True,True,...,True,True,True,True,True,True,NaN,NaN,NaN,NaN
2022-09-24,NaN,NaN,NaN,NaN,True,NaN,NaN,True,True,True,...,True,True,True,True,True,True,NaN,NaN,NaN,NaN
2022-09-25,NaN,NaN,NaN,NaN,True,NaN,NaN,True,True,True,...,True,True,True,True,True,True,NaN,NaN,NaN,NaN
2022-09-26,NaN,NaN,NaN,NaN,True,NaN,NaN,True,True,True,...,True,True,True,True,True,True,NaN,NaN,NaN,NaN


In [9]:
gvkeys = ", ".join([f"'{x}'" for x in components.columns])

In [15]:
sql = f"select gvkey, datadate, tic, prccd, prchd, prcod, ajexdi, trfd  from comp.secd where gvkey in ({gvkeys}) and datadate >= '1995-01-31'"

In [16]:
prices = db.raw_sql(sql)

In [18]:
prices.columns

Index(['gvkey', 'datadate', 'tic', 'prccd', 'prchd', 'prcod', 'ajexdi',
       'trfd'],
      dtype='object')

In [20]:
prices["adjclose"] = prices["prccd"]/prices["ajexdi"]

In [28]:
import pickle

In [30]:
prices["datadate"].max()

datetime.date(2023, 3, 14)

In [32]:
prices[prices["datadate"] >= datetime(2022, 9, 24)]

TypeError: can't compare datetime.datetime to datetime.date

In [33]:
pickle.dump(prices, open("data/prices.p", "wb"))

In [ ]:
datetime(2022, 9, 24)

In [26]:
prices_pivot = pd.pivot_table(prices, index="datadate", columns="gvkey", values="adjclose")

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
prices

In [ ]:
import data.download_yf_data as yf_download
from data.pull_data import pull_yfinance_sample_data
from mom_trans.data_prep import deep_momentum_strategy_features

%load_ext autoreload
%autoreload 2

In [ ]:
# yf_download.main(index_symbol="NDX")

In [ ]:
deep_momentum_strategy_features(pull_yfinance_sample_data("AAPL"))